In [7]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from deepbrain import Extractor
import os
import SimpleITK as sitk

In [3]:
filepath_brain = 'data/miccai_brain/'
outfile_brain = 'data/miccai_stripped_brain'

In [4]:
nib.Nifti1Header.quaternion_threshold = -7e-07

In [5]:
def crop(vol):
    h,w,slices = vol.shape
    crop_vol = np.zeros((152,152,slices))
    for j in range(slices):
        crop_slice = vol[74:226,62:214,j]
        crop_vol[:,:,j] = crop_slice
    return crop_vol
        

def skull_strip(filepath_brain,out_path):
    
    for file in next(os.walk(filepath_brain))[2]:
        nii_vol = nib.load(filepath_brain+file)
        nii_img = nii_vol.get_fdata()
        nii_aff = nii_vol.affine
        ext = Extractor()
        prob = ext.run(nii_img) 
        mask = prob > 0.5
        stripped_image = np.copy(nii_img) # define the masked image as the full image
        stripped_image[mask==False] = 0 
        
        #cropped_vol = crop(stripped_image)
        
        new_nii_vol = nib.Nifti1Image(stripped_image,nii_aff)
        nib.save(new_nii_vol,out_path+file)
        

In [16]:
def n4correction(filepath_brain,out_path):
    
    for file in next(os.walk(filepath_brain))[2]:
        input_image = sitk.ReadImage(filepath_brain+file)
        input_image = sitk.Cast( input_image, sitk.sitkFloat32 )
        corrector = sitk.N4BiasFieldCorrectionImageFilter()
        numberFittingLevels = 4
        maskImage = sitk.OtsuThreshold( input_image, 0, 1, 200 )
        output = corrector.Execute( input_image, maskImage )
        sitk.WriteImage( output, out_path+file )
        
        

        

In [15]:
skull_strip(filepath_brain,outfile_brain)

In [18]:
strip_path = 'data/miccai_stripped_brain/'
out_n4_path = 'data/miccai_normalized'

In [19]:
n4correction(strip_path,out_n4_path)